In [2]:
import talib
import sqlite3
import pandas as pd
from datetime import datetime
from importlib import reload
import time
import numpy as np
import matplotlib.pyplot as plt
import connectorx as cx

In [77]:
import functions; reload(functions)
from functions import *

In [3]:
import sys
print(sys.executable)

c:\Users\Janko\Miniconda3\envs\stonks\python.exe


In [24]:
conn = sqlite3.connect('main.db')
c = conn.cursor()

In [78]:
maxtime = 'SELECT MAX(time) FROM stocks;'
c.execute(maxtime).fetchone()[0]

1720468800000

In [40]:
query = '''
SELECT * FROM stocks
WHERE ticker in (
	SELECT ticker FROM stocks
	WHERE time = (
		SELECT MAX(time) FROM stocks
	)
	ORDER BY volume * vw DESC
	LIMIT 5000
)
'''

In [69]:
bla = '''
SELECT ticker, close, time FROM stocks
WHERE ticker in (
SELECT ticker FROM stocks
	WHERE time = (
		SELECT MAX(time) FROM stocks
	)
    ORDER BY volume * vw DESC
    LIMIT 5000
  )
'''

In [41]:
data = get_data(query)

In [70]:
cxtest = cx.read_sql('sqlite://main.db', bla)

In [76]:
cx2 = cx.read_sql('sqlite://main.db', get_top_stocks())

In [35]:
test = pd.read_sql_query(query, conn)

In [38]:
test2 = pd.read_sql_query(query, conn)

In [16]:
timestamp_to_date(get_data(maxtime)[0][0])

'2024-07-03'

In [28]:
df = pd.read_sql_query(get_top_stocks(10000), conn)

In [17]:
df = get_data(get_top_stocks(10))

In [19]:
df = pd.DataFrame(df)

In [ ]:
onlylast = '''
    SELECT ticker FROM stocks
	WHERE time = (
		SELECT MAX(time) FROM stocks
	)
	ORDER BY volume * vw DESC
    LIMIT 1000;
'''

query = f'''
    SELECT * 
    FROM stocks 
    WHERE time > {timestamp_from_datestring('2024-01-01')}
    AND n > 1000
    '''
	
### test table
beginning = date_to_timestamp('2023-01-01')
end = date_to_timestamp('2024-06-01')
query = f'''
    SELECT * 
    FROM stocks 
    WHERE time > {beginning} AND time < {end}
'''
test = pd.read_sql_query(query, conn)
test.to_sql('test', conn, if_exists='replace', index=False)

In [6]:
def get_data(query):
    c.execute(query)
    data = c.fetchall()
    return data

In [56]:
timestamps = pd.read_sql_query(f'SELECT DISTINCT time FROM {tablename}', conn)
dates = [date_from_timestamp(ts) for ts in timestamps['time']]
trading_days = get_trading_days(from_date=min(dates))[1:]
missingdates = missingdates(dates, trading_days)